# pipeVibSim Example

This notebook demonstrates how to use the `pipeVibSim` library to perform a vibration analysis of a 3D pipe.

In [ ]:
import numpy as np
from pipeVibSim.pipe import Pipe
from pipeVibSim.materials import get_material_properties
from pipeVibSim.simulation import VibrationAnalysis
import pipeVibSim.postprocessing as post

## 1. Define Pipe Geometry

In [ ]:
points = np.array([
    [0, 0, 0],    # origin
    [1, 0, 0],    # 1st virtual point
    [1, 0, 1],    # 2nd virtual point
    [1, 1, 1]     # last point
], dtype=float) * 0.1

pipe = Pipe(points, radius=0.03, step=0.01)

## 2. Define Material Properties

In [ ]:
material_properties = get_material_properties(
    E=110.0e9,      # Young's modulus (Pa)
    rho=8960.0,     # Density (kg/m^3)
    nu=0.34,        # Poisson's ratio
    D_out=0.01,     # Outer diameter (m)
    D_in=0.008,     # Inner diameter (m)
    n_elements=pipe.node_positions.shape[0] - 1
)

## 3. Perform Vibration Analysis

In [ ]:
analysis = VibrationAnalysis(pipe, material_properties)

### Eigensolution

In [ ]:
shapes = analysis.run_eigensolution(maximum_frequency=4000)
print(shapes)

### Plot Mode Shapes

In [ ]:
post.plot_mode_shapes(analysis.geometry, shapes)

### Frequency Response Function (FRF)

In [ ]:
frequencies = np.linspace(0., 500, 1000)
frf = analysis.run_frf(frequencies, load_dof_indices=-4, response_dof_indices=-4)

### Plot FRF

In [ ]:
post.plot_frf(frf)